In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import lightgbm as lgb
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import logging
from torch.utils.data import DataLoader, TensorDataset

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bidding_imbalance_price = pd.read_csv('bidding_training_predictions_imbalance_price.csv')
df_bidding_imbalance_price = df_bidding_imbalance_price[["timestamp_utc","imbalance_price_predictions"]]
df_bidding_imbalance_price.timestamp_utc = pd.to_datetime(df_bidding_imbalance_price.timestamp_utc)
df_bidding_day_ahead_price = pd.read_csv('bidding_training_predictions_day_ahead_price.csv')
df_bidding_day_ahead_price = df_bidding_day_ahead_price[["timestamp_utc","day_ahead_price_predictions"]]
df_bidding_day_ahead_price.timestamp_utc = pd.to_datetime(df_bidding_day_ahead_price.timestamp_utc)
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding["national_demand"] = df_bbidding["ND"].rename("national_demand")
df_bbidding["transmission_system_demand"] = df_bbidding["TSD"].rename("transmission_system_demand")

In [13]:
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc).dt.tz_localize("UTC")
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [14]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price']
    Target_MW = row['Target_MW']
    imbalance_price = row['imbalance_price']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_optimal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))



In [15]:
df_bbidding.revenue_optimal.mean()

56634.21239494943

In [16]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['1']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade_prediction'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal_prediction'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade_prediction'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade_prediction']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade_prediction']))


In [17]:
df_bbidding.revenue_normal_prediction.mean()

33733.555608540315

In [18]:
df_bbidding["trade_residuals"] = df_bbidding["optimized_trade"] - df_bbidding["optimized_trade_prediction"]
y = df_bbidding["trade_residuals"]
X = df_bbidding[["day_ahead_price_predictions","imbalance_price_predictions","optimized_trade_prediction","cos_hour","cos_day", '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',"national_demand","transmission_system_demand"]]

In [9]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import numpy as np

# # Convert data to tensors
# class TradeDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = torch.FloatTensor(X)
#         self.y = torch.FloatTensor(y.values.reshape(-1, 1))
        
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# # Define the MLP model
# class MLP(nn.Module):
#     def __init__(self, input_size):
#         super(MLP, self).__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(input_size, 128),
#             nn.SELU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, 256),
#             nn.SELU(),
#             nn.Dropout(0.2),
#             nn.Linear(256, 64),
#             nn.SELU(),
#             nn.Linear(64, 1)
#         )
        
#     def forward(self, x):
#         return self.layers(x)

# # Prepare the data
# scaler1 = StandardScaler()
# X_scaled = scaler1.fit_transform(X)

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# # Create data loaders
# train_dataset = TradeDataset(X_train, y_train)
# test_dataset = TradeDataset(X_test, y_test)

# batch_size = 512
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)

# # Initialize model, loss function, and optimizer
# model = MLP(input_size=X.shape[1])
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 1000
# train_losses = []
# test_losses = []

# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0
#     for batch_X, batch_y in train_loader:
#         optimizer.zero_grad()
#         outputs = model(batch_X)
#         loss = criterion(outputs, batch_y)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()
    
#     # Calculate average training loss
#     train_loss = train_loss / len(train_loader)
#     train_losses.append(train_loss)
    
#     # Validation
#     model.eval()
#     test_loss = 0
#     with torch.no_grad():
#         for batch_X, batch_y in test_loader:
#             outputs = model(batch_X)
#             loss = criterion(outputs, batch_y)
#             test_loss += loss.item()
    
#     test_loss = test_loss / len(test_loader)
#     test_losses.append(test_loss)
    
#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

# # Plot training curves
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# plt.plot(train_losses, label='Training Loss')
# plt.plot(test_losses, label='Test Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Test Loss over Time')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Evaluate final model
# model.eval()
# with torch.no_grad():
#     X_test_tensor = torch.FloatTensor(X_test)
#     y_pred = model(X_test_tensor).numpy()
    
# # Calculate metrics
# from sklearn.metrics import mean_squared_error, r2_score
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'\nFinal Results:')
# print(f'MSE: {mse:.4f}')
# print(f'RMSE: {np.sqrt(mse):.4f}')
# print(f'R2 Score: {r2:.4f}')

# # Plot predictions vs actual values
# plt.figure(figsize=(10, 6))
# plt.scatter(y_test, y_pred, alpha=0.5)
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
# plt.xlabel('Actual Values')
# plt.ylabel('Predicted Values')
# plt.title('Predicted vs Actual Values')
# plt.grid(True)
# plt.show()

In [10]:
# torch.save(model.state_dict(), 'trade_residuals_model.pth')

In [11]:


logging.basicConfig(level=logging.INFO)

def objective(trial, X_train, X_test, y_train, y_test):
    """
    Objective function for Optuna optimization
    """
    params = {
        'objective': 'regression',
        'metric': 'l2',
        'boosting_type': 'gbdt',
        'verbose': -1,
        
        # Parameters to optimize
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 200),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 10.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
    }

    # Create datasets
    train_data = lgb.Dataset(X_train, y_train)
    valid_data = lgb.Dataset(X_test, y_test, reference=train_data)

    # Train model with callbacks
    model = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=20),
            lgb.log_evaluation(period=100)
        ]
    )

    # Predict and calculate loss
    y_pred = model.predict(X_test)
    loss = mean_squared_error(y_test, y_pred)

    # Log progress
    logging.info(f"Trial {trial.number} - MSE: {loss:.4f}")

    return loss

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and optimize study
study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: objective(trial, X_train, X_test, y_train, y_test),
    n_trials=30
)

# Get best parameters and train final model
best_params = study.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'l2'
best_params['boosting_type'] = 'gbdt'
best_params['verbose'] = -1

print("\nBest parameters:")
for key, value in best_params.items():
    print(f"    {key}: {value}")

# Train final model with best parameters
train_data = lgb.Dataset(X_train, y_train)
valid_data = lgb.Dataset(X_test, y_test, reference=train_data)

final_model = lgb.train(
    best_params,
    train_data,
    valid_sets=[valid_data],
    num_boost_round=1000,
    callbacks=[
        lgb.early_stopping(stopping_rounds=20),
        lgb.log_evaluation(period=100)
    ]
)

# Make predictions and evaluate
y_pred = final_model.predict(X_test)
final_mse = mean_squared_error(y_test, y_pred)
print(f"\nFinal MSE: {final_mse:.4f}")

# Calculate feature importance
importance = final_model.feature_importance(importance_type='gain')
feature_names = final_model.feature_name()

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values('Importance', ascending=False)

print("\nTop 20 Most Important Features:")
print(feature_importance.head(20))

[I 2024-10-27 16:18:30,745] A new study created in memory with name: no-name-61460379-727c-4563-ad54-2af59c1c2634


Training until validation scores don't improve for 20 rounds
[100]	valid_0's l2: 131009
[200]	valid_0's l2: 126565
[300]	valid_0's l2: 124780


INFO:root:Trial 0 - MSE: 123673.1544
[I 2024-10-27 16:18:32,326] Trial 0 finished with value: 123673.15440073874 and parameters: {'num_leaves': 113, 'learning_rate': 0.053648763636348726, 'max_depth': 63, 'min_child_samples': 42, 'subsample': 0.8891885063015936, 'colsample_bytree': 0.8932119327455481, 'reg_alpha': 0.890937477014508, 'reg_lambda': 0.1594730622858308, 'min_split_gain': 0.43710362076589826, 'min_child_weight': 9.141817473417317, 'bagging_freq': 9}. Best is trial 0 with value: 123673.15440073874.


[400]	valid_0's l2: 123731
Early stopping, best iteration is:
[386]	valid_0's l2: 123673
Training until validation scores don't improve for 20 rounds
[100]	valid_0's l2: 140081
[200]	valid_0's l2: 133633
[300]	valid_0's l2: 130560
[400]	valid_0's l2: 128369
[500]	valid_0's l2: 126848
[600]	valid_0's l2: 126031
[700]	valid_0's l2: 125357
[800]	valid_0's l2: 124718
[900]	valid_0's l2: 124194
[1000]	valid_0's l2: 123767
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 123767


INFO:root:Trial 1 - MSE: 123766.8993
[I 2024-10-27 16:18:35,425] Trial 1 finished with value: 123766.89932001715 and parameters: {'num_leaves': 98, 'learning_rate': 0.023881323053088738, 'max_depth': 197, 'min_child_samples': 39, 'subsample': 0.7794821105644396, 'colsample_bytree': 0.5749361104258286, 'reg_alpha': 0.046739319558892434, 'reg_lambda': 0.7594311635012125, 'min_split_gain': 0.6793093216714032, 'min_child_weight': 9.460705337275284, 'bagging_freq': 1}. Best is trial 0 with value: 123673.15440073874.


...

Early stopping, best iteration is:
[470]	valid_0's l2: 121280
Training until validation scores don't improve for 20 rounds
[100]	valid_0's l2: 138975
[200]	valid_0's l2: 132925
[300]	valid_0's l2: 130268
[400]	valid_0's l2: 128095


INFO:root:Trial 29 - MSE: 126661.9063


[500]	valid_0's l2: 126807
Early stopping, best iteration is:
[526]	valid_0's l2: 126662


[I 2024-10-27 16:20:17,739] Trial 29 finished with value: 126661.90628200864 and parameters: {'num_leaves': 69, 'learning_rate': 0.029989752256810068, 'max_depth': 57, 'min_child_samples': 39, 'subsample': 0.8393310535403973, 'colsample_bytree': 0.883889390474142, 'reg_alpha': 0.5547926208027394, 'reg_lambda': 0.18770291402603573, 'min_split_gain': 0.4793332247689454, 'min_child_weight': 7.117707062789186, 'bagging_freq': 10}. Best is trial 20 with value: 118775.83297392583.



Best parameters:
    num_leaves: 135
    learning_rate: 0.019601200692247408
    max_depth: 44
    min_child_samples: 11
    subsample: 0.9632509242021468
    colsample_bytree: 0.9957221221030387
    reg_alpha: 0.2237659358045655
    reg_lambda: 0.37944705979377186
    min_split_gain: 0.5334627843067469
    min_child_weight: 3.7673478641972493
    bagging_freq: 2
    objective: regression
    metric: l2
    boosting_type: gbdt
    verbose: -1
Training until validation scores don't improve for 20 rounds
[100]	valid_0's l2: 138006
[200]	valid_0's l2: 130644
[300]	valid_0's l2: 126742
[400]	valid_0's l2: 124746
[500]	valid_0's l2: 123287
[600]	valid_0's l2: 122370
[700]	valid_0's l2: 121239
[800]	valid_0's l2: 120092
[900]	valid_0's l2: 119393
[1000]	valid_0's l2: 118776
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 118776

Final MSE: 118775.8330

Top 20 Most Important Features:
                        Feature    Importance
1   imbalance_price_predictions  4.031297

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Best parameters:
#     num_leaves: 281
#     learning_rate: 0.013741984438991241
#     max_depth: 166
#     min_child_samples: 12
#     subsample: 0.7931210792516683
#     colsample_bytree: 0.8058298185354192
#     reg_alpha: 0.9473159380670849
#     reg_lambda: 0.44387394194115004
#     min_split_gain: 0.09221192058768873
#     min_child_weight: 7.582890157437369
#     bagging_freq: 5
#     objective: regression
#     metric: l2
#     boosting_type: gbdt
#     verbose: -1
# Training until validation scores don't improve for 20 rounds
# [100]	valid_0's l2: 140567
# [200]	valid_0's l2: 130122
# [300]	valid_0's l2: 125873
# [400]	valid_0's l2: 123747
# [500]	valid_0's l2: 121908
# [600]	valid_0's l2: 120801
# [700]	valid_0's l2: 119977
# ...
# 11                            7  7.584879e+09
# 8                             4  7.275663e+09
# 10                            6  7.180459e+09
# 9                             5  6.446710e+09

In [20]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import joblib

# Create dataset for LightGBM
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# Define updated parameters
params = {
    'objective': 'regression',
    'metric': 'l2',
    'boosting_type': 'gbdt',
    'num_leaves': 281,
    'learning_rate': 0.013741984438991241,
    'max_depth': 166,
    'min_child_samples': 12,
    'subsample': 0.7931210792516683,
    'colsample_bytree': 0.8058298185354192,
    'reg_alpha': 0.9473159380670849,
    'reg_lambda': 0.44387394194115004,
    'min_split_gain': 0.09221192058768873,
    'min_child_weight': 7.582890157437369,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the model with early stopping
gbm = lgb.train(params,
                train_set=lgb_train,
                num_boost_round=1000,  # Set high to enable early stopping
                valid_sets=[lgb_eval],  # Use test set for early stopping validation
                callbacks=[lgb.early_stopping(stopping_rounds=20)])  # Early stopping callback set to 20 rounds

# Predict on test set
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Save the model using joblib
joblib.dump(gbm, 'lightgbm_model.joblib')
print("Model saved as lightgbm_model_best_params.joblib")


Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 113067
Mean Squared Error: 113066.64753952342
Model saved as lightgbm_model_best_params.joblib


In [21]:
predictions1 = gbm.predict(X)
df_bbidding["trade_residuals_predictions"] = predictions1 + df_bbidding["optimized_trade_prediction"]

In [22]:
df_bbidding['revenue_normal_prediction'] = df_bbidding['day_ahead_price'] * df_bbidding['trade_residuals_predictions'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['trade_residuals_predictions']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['trade_residuals_predictions']))

In [23]:
df_bbidding.revenue_normal_prediction.mean()

50049.91730051513

In [24]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [25]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))
path_df = os.path.abspath(os.path.join(current_dir, '..'))
df_demand = pd.read_csv(os.path.join(path_df, 'day_ahead_demand_forecast.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [26]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)


In [27]:
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new.timestamp = pd.to_datetime(df_api_new.timestamp)

In [28]:

# Create a continuous time series from the minimum to maximum timestamp at 30-minute intervals
full_timestamp_range = pd.date_range(start=df_api_new['timestamp'].min(), end=df_api_new['timestamp'].max(), freq='30min')
# Reindex the dataframe using the full range of timestamps
df_api_new_1 = df_api_new.set_index('timestamp').reindex(full_timestamp_range, method=None)
df_api_new_1 = df_api_new_1.reset_index().rename(columns={'index': 'timestamp'})
# Create the 'prediction_date' column based on the timestamp
df_api_new_1['prediction_date'] = df_api_new_1['timestamp'].dt.date

In [29]:
df_demand = df_demand[["timestamp_utc","transmission_system_demand","national_demand"]]
df_demand = df_demand.rename(columns={'timestamp_utc': 'timestamp_utc_demand'})

In [31]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_demand.timestamp_utc_demand = pd.to_datetime(df_demand.timestamp_utc_demand)
df_api_new_merged = pd.merge(df_api_new_1,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_demand, left_on='timestamp', right_on='timestamp_utc_demand', how='left',suffixes=('_x', '_y'))
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)


In [32]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price","national_demand","transmission_system_demand"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(192)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(96)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()


In [33]:
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
wind_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/wind_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)

In [34]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,solar_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = pd.merge(df_api_new_merged2,wind_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = df_api_new_merged2.rename(columns={
    "generation_mw_x": "generation_solar",
    "generation_mw_y": "generation_wind"
})
df_api_new_merged2 = df_api_new_merged2.groupby("timestamp_utc").last().reset_index()


In [35]:
import pickle
df_api_new_merged2_X = df_api_new_merged2[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9","national_demand","transmission_system_demand"]].copy()

scaler_path = "LSTM_imbalance_scaler.pkl"
# Laden des StandardScalers aus der Datei
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

# Skalieren der Daten
df_api_new_merged2_X_scaled = scaler.transform(df_api_new_merged2_X)

# Konvertieren der Daten in PyTorch-Tensoren
X_test = torch.tensor(df_api_new_merged2_X_scaled, dtype=torch.float32)
X_test = X_test.unsqueeze(1)  # Adds a sequence length dimension

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [37]:
input_size = 17  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_imbalance_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["imvalance_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_27468\2878417824.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_imbalance_p

In [38]:
input_size = 17  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["day_ahead_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_27468\2156771396.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_p

In [39]:
df_api_new_merged2["Total_MW"] = df_api_new_merged2["generation_solar"] + df_api_new_merged2["generation_wind"]

In [40]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price']
    Target_MW = row['Total_MW']
    imbalance_price = row['imbalance_price']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade_perfect'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal_perfect'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade_perfect'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_perfect']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_perfect']))


In [41]:
df_api_new_merged2.revenue_normal_perfect.mean()

36591.038625752415

In [42]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['1']
    imbalance_price = row['imvalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']))


In [43]:
df_api_new_merged2.revenue_normal.mean()

21974.935846313318

In [44]:
df_api_new_merged2
X = df_api_new_merged2[["day_ahead_price_predictions","imvalance_price_predictions","optimized_trade","cos_hour","cos_day",'1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',"national_demand","transmission_system_demand"]]
predictions = gbm.predict(X)
df_api_new_merged2["newly_optimized_trade"] = predictions + df_api_new_merged2["optimized_trade"]

In [45]:
df_api_new_merged2['revenue_normal_new'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['newly_optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['newly_optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['newly_optimized_trade']))

In [46]:
df_api_new_merged2.revenue_normal_new.mean()

23570.79750907368

In [38]:
#üplot the columns  newly_optimized_trade optimized_trade optimized_trade_perfect
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged2.timestamp_utc, y=df_api_new_merged2.newly_optimized_trade, mode='lines', name='newly_optimized_trade'))
fig.add_trace(go.Scatter(x=df_api_new_merged2.timestamp_utc, y=df_api_new_merged2.optimized_trade, mode='lines', name='optimized_trade'))
fig.add_trace(go.Scatter(x=df_api_new_merged2.timestamp_utc, y=df_api_new_merged2.optimized_trade_perfect, mode='lines', name='optimized_trade_perfect'))
fig.show()

In [39]:
# X_scaled = scaler1.transform(X.values)
# X_scaled = torch.tensor(X_scaled, dtype=torch.float32)

In [40]:
# model.eval()
# with torch.no_grad():
#     predictions_mlp = model(X_scaled)
# predictions_mlp = predictions_mlp.numpy()
# predictions_mlp.shape

In [41]:
# (predictions_mlp + df_api_new_merged2[["optimized_trade"]]).shape

In [42]:
# df_api_new_merged2["optimized_trade_prediction_mlp"] = predictions_mlp + df_api_new_merged2[["optimized_trade"]]

In [43]:
df_api_new_merged2['revenue_normal_new'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade_prediction_mlp'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_prediction_mlp']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_prediction_mlp']))

KeyError: 'optimized_trade_prediction_mlp'

In [91]:
df_api_new_merged2.revenue_normal_new.mean()

2047.165563438268